<a href="https://colab.research.google.com/github/rg-smith/remote-sensing-hydro-2026/blob/main/assignments/homework2/homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2

In this lab, you will use python (and a module called Google Earth Engine) to view satellite imagery, and access and compare precipitation and ET datasets.

Before starting the sections, we will import the modules we need, initialize Google Earth Engine, and define some functions.

The first block of code only needs to be run once, the first time you open your session (if you close out of the session and open again, you will need to run this again). Follow the prompts to initialize earth engine. You will be taken to a link where you need to give permission to link your google account with Google Earth Engine, then copy and paste some text below the code block.

As you go through this assignment, you will be prompted to answer questions. Record the answers in a word doc. When you are finished, download this notebook as a .ibynb file, and submit that and the word doc.

In [ ]:
!pip install geemap

In [ ]:
import ee
import folium
import numpy as np
import branca.colormap as cm
import requests
import zipfile
import os
from tqdm import tqdm
import geemap

If you have not registered for Google Earth Engine yet, do so at https://code.earthengine.google.com/. At the top-right corner, you will see a name next to your user icon. Copy that name and past where 'your project name' is located in the code block below. NOTE: if you see any warning symbols on the [Google Earth Engine](https://code.earthengine.google.com) website next to your profile icon, click it and resolve it before proceeding.

In [ ]:
#if not ee.data._credentials:
ee.Authenticate()
ee.Initialize(project='paste your code here')

This block of code also only needs to be run once. It defines many useful functions, some of which you will use in this exercise. The most important thing for you to understand is what arguments are needed to run the functions, and what their outputs are. You will use some of them in this exercise, and some later on!

# Define custom functions

In [ ]:
# functions needed for this lab (and some other useful ones that you can use if you're interested)

# to convert a google earth engine image to a python array
def to_array(img,aoi):
  band_arrs = img.sampleRectangle(region=aoi,properties=['scale=1000'],defaultValue=-999)

  band_names=img.bandNames().getInfo()

  for kk in range(len(band_names)):
    if kk==0:
      dat1=np.array(band_arrs.get(band_names[kk]).getInfo())
      dat_full=np.zeros((dat1.shape[0],dat1.shape[1],len(band_names)))
      dat_full[:,:,kk]=dat1
    else:
      dat=np.array(band_arrs.get(band_names[kk]).getInfo())
      dat_full[:,:,kk]=dat
  return(dat_full)

# to calculate an index
def getIndex(image,b1,b2):
  return image.normalizedDifference([b1, b2])

# to calculate a ratio
def getRatio(image1,image2):
  ratio=image1.divide(image2)
  return ratio

# to create a color map from a specific image
def getVisparams(image,aoi):
  range = image.reduceRegion(ee.Reducer.percentile([1, 99]),aoi,300)
  vals = range.getInfo()
  min=list(vals.items())[0][1]
  max=list(vals.items())[1][1]
  visParams = {'min': min, 'max': max, "palette": ["red", "orange", "yellow", "cyan", "blue"]}
  return(visParams)

# to get the link to download an earth engine image
def getLink(image,aoi):
  link = image.getDownloadURL({
    'scale': 1000,
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'region': aoi})
  print(link)

# create an earth engine geometry polygon
def addGeometry(min_lon,max_lon,min_lat,max_lat):

  geom = ee.Geometry.Polygon(
      [[[min_lon, max_lat],
        [min_lon, min_lat],
        [max_lon, min_lat],
        [max_lon, max_lat]]])
  return(geom)

# to export an image to google drive
def export_to_drive(raster,filename,foldername,geometry):
  # Export the image, specifying scale and region.
  task = ee.batch.Export.image.toDrive(**{
      'image': raster,
      'description': filename,
      'folder': foldername,
      'fileNamePrefix': filename,
      'scale': 1000,
      'region': geometry,
      'fileFormat': 'GeoTIFF',
      'formatOptions': {
        'cloudOptimized': 'true'
      },
  })
  task.start()

def get_imgcollection(date1,date2,geometry,collection_name,band_name,function='mean'):
  collection = ee.ImageCollection(collection_name)
  if function=='mean':
      img = collection.filterDate(date1,date2).select(band_name).mean().clip(geometry)
  if function=='sum':
      img = collection.filterDate(date1,date2).select(band_name).sum().clip(geometry)
  return(img)

def get_img(geometry,collection_name,band_name):
  img = ee.Image(collection_name).select(band_name).clip(geometry)
  return(img)

# to get the link to download an earth engine image
def getLink(image,fname,aoi,scale=1000):
  link = image.getDownloadURL({
    'scale': scale,
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'region': aoi,
    'name': fname})
  # print(link)
  return(link)

def download_img(img,geom,fname,scale=1000):
    linkname = getLink(img,fname,geom,scale=scale)
    response = requests.get(linkname, stream=True)
    zipped = fname+'.zip'
    with open(zipped, "wb") as handle:
        for data in tqdm(response.iter_content()):
            handle.write(data)

    with zipfile.ZipFile(zipped, 'r') as zip_ref:
        zip_ref.extractall('')
    os.remove(zipped)


# Part 1: Viewing ET data for a given geometry and date



First, we will define the study area and time period and load some PRISM, GPM and elevation data. After running this code block, go to [Google Earth Engine](https://code.earthengine.google.com) and use the search bar at the top to find each of the datasets that you loaded (the fourth argument each time the 'get_imgcollection' function is called). Briefly describe in your lab report what the source of each dataset is.

In [ ]:
start = '2021-10-01'
end = '2022-09-30'

geom = addGeometry(-106, -104,40,41) # min long, max long, min lat, max lat

prism_img = get_imgcollection(start,end,geom,'OREGONSTATE/PRISM/ANm','ppt','sum')
vis_prism = getVisparams(prism_img,geom) # create visualization for precip

ET_img = get_imgcollection(start,end,geom,'OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0','et_ensemble_mad','sum')
vis_ET = getVisparams(ET_img,geom) # create visualization for ET

ETo = get_imgcollection(start,end,geom,'IDAHO_EPSCOR/GRIDMET','eto','sum')
vis_ret = getVisparams(ETo,geom) # create visualization for reference ET

Now we will compare water year ET to water year precipitation. Areas with much higher ET than precipitation could receive water from streams, groundwater, or irrigation. They could also represent regions with errors in the precipitation or ET estimates.

We will also divide water year actual ET by water year reference ET.

The last line displays the map. Scroll to the top so you can check/uncheck layers. Note that the bottom layer on the legend will show up on top. Take a screenshot for your lab report.

Comment on discrepancies between ET and precipitation. Group them into discrepancies that you think have a physical meaning, and discrepancies that you think may be related to noise in the data.

In [ ]:
Map = geemap.Map(center=[40.5,-105], zoom=9)
Map.add_basemap("HYBRID")

Map.addLayer(prism_img, vis_prism, "PRISM Annual Precip")
Map.add_colorbar(vis_prism, label="PRISM precip (mm)", layer_name='PRISM Annual Precip')
Map.addLayer(ET_img,vis_ET,'OpenET Annual ET')
Map.add_colorbar(vis_ET,label='OpenET ET (mm)',layer_name='OpenET Annual ET')
Map.addLayer(RET,vis_ret,'GRIDMET Annual Reference ET')
Map.add_colorbar(vis_ret,label='GRIDMET ETo (mm)',layer_name='GRIDMET Annual Reference ET')

Map.addLayer(geom, {},"Study area")

Map.add_inspector()

Map #visualize the map

Using the get_ratio function in the 'Define custom functions' section, create a ratio of actual ET to reference ET, and display that in a map following the pattern of the code below. Comment on the ratio (actual ET/reference ET). What does that ratio physically represent?

In [ ]:
Map2 = geemap.Map(center=[40.5,-105], zoom=9)
ET_ETo = ???
vis_ET_ETR = ??
Map2.addLayer(???)
Map2.add_colorbar(???)

Map2.add_inspector()
Map2 #display map

We will now make a map of the land use types from the USDA Cropland Data Layer

In [ ]:
CDL = get_imgcollection(start,end,geom,'USDA/NASS/CDL','cropland')
vis_CDL = getVisparams(CDL,geom)

Map3 = geemap.Map(center=[40.5,-105], zoom=9)
Map3.addLayer(CDL,vis_CDL,'USDA Cropland Data Layer')
Map3.add_colorbar(vis_CDL,label='USDA Cropland Data Layer',layer_name='USDA Cropland Data Layer')

Map3.add_inspector()
Map3

Now, we will export the tiffs to your notebook folder. We will then use some other python functions to do analysis.

In [ ]:
download_img(ET_img,geom,'lab2_ET_CO',scale=100) #specifies a resolution of 100 m
download_img(prism_img,geom,'lab2_prism_precip_CO',scale=100)
download_img(ETo,geom,'lab2_ETo_CO',scale=100)
download_img(CDL,geom,'lab2_CDL_CO',scale=100) #specifies a resolution of 100 m

Now, we will load the rasters we just downloaded with a separate python package--rasterio--and compare ET with ETo.

In [ ]:
!pip install rasterio
import rasterio
from rasterio.warp import reproject, Resampling
import matplotlib.pyplot as plt

Click on the 'Files' tab on the left. Go to drive > MyDrive > Colab Notebooks. You should see some '.tif' files for prism, gpm and elevation data. If you don't see them yet, you may have to wait a few minutes for them to get saved to the cloud.

Now you will load the rasters into python with rasterio. This package works more easily with the usual python packages, making plotting easier.

In [ ]:
ET=rasterio.open('/content/lab2_ET_CO.et_ensemble_mad.tif')
prism=rasterio.open('/content/lab2_prism_precip_CO.ppt.tif')
ETo=rasterio.open('/content/lab2_ETo_CO.eto.tif')
CDL=rasterio.open('/content/lab2_CDL_CO.cropland.tif')

ET_flat=ET.read(1).flatten()
prism_flat=prism.read(1).flatten()
ETo_flat=ETo.read(1).flatten()
CDL_flat=CDL.read(1).flatten()


In [ ]:
# ignore this section unless you are getting errors related to the size of rasters.

# define a function that reprojects a raster that is loaded in memory using rasterio
# def reproject_rasterio(src,dest):
#     dat,transform=reproject(src.read(1),dest.read(1),
#               src_crs=src.crs,
#               src_transform=src.transform,
#               dst_crs=dest.crs,dst_resolution=dest.res,
#               dst_transform=dest.transform,
#               resampling=Resampling.nearest,
#               dst_nodata=0)
#     return(dat,transform)

# # reproject all rasters to ET crs, extent and cell size
# prism_reproj,tr=reproject_rasterio(prism,ET)
# ETo_reproj,tr=reproject_rasterio(ETo,ET)
# CDL_reproj,tr=reproject_rasterio(CDL,ET)

# ET_flat=ET.read(1).flatten()
# prism_flat=prism.read(1).flatten()
# ETo_flat=ETo.read(1).flatten()
# CDL_flat=CDL.read(1).flatten()

Now, we will make a crossplot of reference ET and ET, colored by precipitation.

In [ ]:
plt.figure(figsize=(12,8));plt.scatter(ETo_flat,ET_flat,c=prism_flat,s=1,alpha=0.3)
plt.plot([0,1500],[0,1500],'r-')
plt.xlabel('GRIDMET ETo, mm')
plt.ylabel('OpenET ET, mm')
plt.colorbar()

Note the difference in ETo and actual ET. Why do you think a small subset of the points are very close to ETo?

In [ ]:
# now we view correlation for evergreen forest (CDL value of 142)
plt.figure(figsize=(12,8));plt.scatter(ETo_flat[CDL_flat==142],ET_flat[CDL_flat==142],c=prism_flat[CDL_flat==142],s=1,alpha=0.3)
plt.plot([0,1500],[0,1500],'r-')
plt.xlabel('GRIDMET ETo, mm')
plt.ylabel('OpenET ET, mm')
plt.colorbar()

Comment on how precipitation is influencing the relationship between ETo and actual ET in evergreen forests.

In [ ]:
# now we compare correlation for corn (CDL value of 1) and alfalfa (CDL value of 36)
plt.figure(figsize=(12,8));plt.hist(ET_flat[CDL_flat==1],color='blue',alpha=0.5)
plt.hist(ET_flat[CDL_flat==36],color='red',alpha=0.5)
plt.legend(['Corn','Alfalfa'])
plt.xlabel('ET, mm')
plt.ylabel('Frequency')

In [ ]:
# now plot the ratio of ET to ETo against precipitation
ETfrac = ET_flat/ETo_flat
plt.figure(figsize=(12,8));plt.scatter(prism_flat,ETfrac,s=1,alpha=0.1)

# create trendline
z = np.polyfit(prism_flat, ETfrac,1)
p = np.poly1d(z)
plt.plot(prism_flat[CDL_flat==142],p(prism_flat[CDL_flat==142]),"r--")

plt.xlabel('PRISM precip, mm')
plt.ylabel('ET/ETo')

In [ ]:
# repeat for evergreen forest
ETfrac = ET_flat/ETo_flat
plt.figure(figsize=(12,8));plt.scatter(prism_flat[CDL_flat==142],ETfrac[CDL_flat==142],s=1,alpha=0.1)

# create trendline
z = np.polyfit(prism_flat[CDL_flat==142], ETfrac[CDL_flat==142],1)
p = np.poly1d(z)
plt.plot(prism_flat[CDL_flat==142],p(prism_flat[CDL_flat==142]),"r--")
plt.xlabel('PRISM precip, mm')
plt.ylabel('ET/ETo')

In [ ]:
# repeat for corn
ETfrac = ET_flat/ETo_flat
plt.figure(figsize=(12,8));plt.scatter(prism_flat[CDL_flat==1],ETfrac[CDL_flat==1],s=1,alpha=0.1)
# add best fit line in red
z = np.polyfit(prism_flat[CDL_flat==1], ETfrac[CDL_flat==1],1)
p = np.poly1d(z)
plt.plot(prism_flat[CDL_flat==1],p(prism_flat[CDL_flat==1]),"r--")

plt.xlabel('PRISM precip, mm')
plt.ylabel('ET/ETo')

What is the relationship between precip and that ratio for different land use types, and why?

# Part 2: Explore!

Pick a different study area. It could be a potential location of your term project, or somewhere else. Recommended to work in CONUS so the above datasets work. Load and plot the datasets as above, then explore the data. Comment on what is driving ET in the area you chose based on the data you see.

You will need to 'fill in the blanks' for the code below.

In [ ]:
# create a bounding box that defines the study area. This will cover the Lake of the Ozarks region
geom_study_area = addGeometry(?,?,?,?) # min long, max long, min lat, max lat

# define dates of interest (inclusive).
start = '?'
end = '?'

# use the tools in the example from Part 1 to pull the same data
# NOTE: it will be easiest to do this within CONUS. If you choose to pick a different
# study area, you will need to use datasets that are available outside of CONUS. There
# are some options, but they are not as good and there's no catch-all. If your study
# area is outside of the US, I recommend that you pick a region within CONUS for this assignment.
# However, if you would like to test your data wrangling skills, feel free to search
# for comparable datasets outside of the US.

In [ ]:
# fill in this and additional codeblocks to repeat the analysis from Part 1.